In [37]:
# 必要なライブラリのインストール
!pip install requests beautifulsoup4 pandas

# 必要なライブラリのインポート
import requests
from bs4 import BeautifulSoup
import time
import sqlite3
import re
import pandas as pd
from IPython.display import display

In [47]:
# スクレイピング対象のGitHub ユーザー名
USERNAME = "Wakana-0907"
# ユーザーのリポジトリ一覧タブのURLを使用
BASE_URL = f"https://github.com/{USERNAME}?tab=repositories"

# 収集データを格納するリスト
scraped_data = []

print("--- スクレイピング開始 ---")
current_url = BASE_URL
print(f"リポジトリページをスクレイピング中... URL: {current_url}")

# ユーザーエージェントを設定
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
}

# Webページを取得
response = requests.get(current_url, headers=headers)

# 課題の指示に従い、1秒待機
time.sleep(1) 

if response.status_code != 200:
    print(f"エラー: HTTPステータスコード {response.status_code} - スクリプトを終了します。")
else:
    # HTML解析
    soup = BeautifulSoup(response.content, 'html.parser')
    
    # リポジトリリストの抽出 (ユーザーページ構造に修正)
    # GitHubのHTML構造に基づき、リポジトリのリスト要素を検索
    repo_list = soup.find_all('li', class_='source') 

    if not repo_list:
        print("リポジトリが見つかりませんでした。公開リポジトリがないか、HTML構造が変更された可能性があります。")
    
    for repo in repo_list:
        # 1. リポジトリ名
        name_tag = repo.find('a', itemprop='name codeRepository')
        repo_name = name_tag.text.strip() if name_tag else "N/A"
        
        # 2. 主要な言語
        lang_tag = repo.find('span', itemprop='programmingLanguage')
        main_language = lang_tag.text.strip() if lang_tag else "N/A"
        
        # 3. スターの数
        star_link = repo.find('a', href=re.compile(r'/stargazers'))
        stars_count = "0"
        if star_link:
            star_span = star_link.find('span', class_='css-truncate-target')
            if star_span:
                stars_count = star_span.text.strip()
            
        scraped_data.append({
            "repository_name": repo_name,
            "main_language": main_language,
            "stars_count": stars_count,
        })
        
print("スクレイピング完了！")

# 結果の確認（DataFrameに変換）
df = pd.DataFrame(scraped_data)
print("\n--- 取得データ (抜粋) ---")
display(df.head())
print(f"\n--- 総リポジトリ数: {len(df)} ---")

--- スクレイピング開始 ---
リポジトリページをスクレイピング中... URL: https://github.com/Wakana-0907?tab=repositories
スクレイピング完了！

--- 取得データ (抜粋) ---


,repository_name,main_language,stars_count
0,dsprog2_2025,N/A,0
1,deprogramming2,N/A,0
2,-,Jupyter Notebook,0
3,dsprog1_2025,Jupyter Notebook,0
4,git-practice-2025,HTML,0



--- 総リポジトリ数: 6 ---


In [48]:
import sqlite3

path = ''
# データベースファイル名を 'github_repos.db' に変更
db_name = 'github_repos.db'
conn = None 

print("--- DBおよび 'repositories' テーブル作成開始 ---")

try:
    # 1. DB接続オブジェクトの作成
    conn = sqlite3.connect(path + db_name)
    print(f"DB '{db_name}' に接続しました。")

    # 2. SQLを実行するためのカーソルオブジェクトを取得
    cur = conn.cursor()
    
    # 3. SQL文の作成（テーブルの作成）
    # テーブル名を 'repositories' に変更し、格納する列を定義
    cur.execute('DROP TABLE IF EXISTS repositories;') 
    sql = 'CREATE TABLE repositories (repository_name TEXT, main_language TEXT, stars_count TEXT);'

    # 4. SQL文の実行
    cur.execute(sql)
    print("'repositories' テーブルを作成しました。")

    # 5. 変更をDBに反映させる
    conn.commit()
    print("コミット完了。")
except sqlite3.Error as e:
    # エラーが発生した場合
    print(f'エラーが発生しました: {e}')
    
finally:
    # 6. DBへの接続を閉じる
    if conn:
        conn.close()
        print("DB接続を閉じました。")

print("\n--- 処理完了 ---")

--- DBおよび 'repositories' テーブル作成開始 ---
DB 'github_repos.db' に接続しました。
'repositories' テーブルを作成しました。
コミット完了。
DB接続を閉じました。

--- 処理完了 ---


In [49]:
import sqlite3

path = ''
# DB名とテーブル名は、前のステップで作成したものを使用
db_name = 'github_repos.db'
table_name = 'repositories' 
conn = None 

print("--- リポジトリデータ挿入開始 ---")

try:
    # 1. DB接続オブジェクトの作成
    conn = sqlite3.connect(path + db_name)
    print(f"DB '{db_name}' に接続しました。")

    # 2. SQLを実行するためのカーソルオブジェクトを取得
    cur = conn.cursor()
    
    # 3. SQL文の作成（複数レコードの挿入）
    # テーブル名と列名 (repository_name, main_language, stars_count) に修正
    sql = f"INSERT INTO {table_name} (repository_name, main_language, stars_count) VALUES (?, ?, ?);"

    # 挿入データリストを作成
    # scraped_data (辞書のリスト) を、executemany が要求するタプルのリストに変換
    data_to_insert = [
        (repo['repository_name'], repo['main_language'], repo['stars_count'])
        for repo in scraped_data
    ]
    
    # 4. SQL文の実行（複数データ挿入）
    cur.executemany(sql, data_to_insert)
    print(f"{len(data_to_insert)} 件のレコードをテーブル '{table_name}' に挿入しました。")

    # 5. 変更をDBに反映させる
    conn.commit()
    print("コミット完了。")
except NameError:
    # scraped_data が定義されていない場合の例外処理
    print("エラー: 'scraped_data' が定義されていません。スクレイピングコードを実行してください。")
except sqlite3.Error as e:
    # SQLiteに関するエラーが発生した場合
    print(f'エラー: DB操作中にエラーが発生しました: {e}')
    
finally:
    # 6. DBへの接続を閉じる
    if conn:
        conn.close()
        print("DB接続を閉じました。")

--- リポジトリデータ挿入開始 ---
DB 'github_repos.db' に接続しました。
6 件のレコードをテーブル 'repositories' に挿入しました。
コミット完了。
DB接続を閉じました。
